# Settings

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

# Loading the data

In [2]:
sample_submission = pd.read_csv("C:/Users/odeli/Desktop/dataCamp_data/sample_submission.csv")
test = pd.read_csv("C:/Users/odeli/Desktop/dataCamp_data/test.csv")
specs = pd.read_csv("C:/Users/odeli/Desktop/dataCamp_data/specs.csv")
train_labels = pd.read_csv("C:/Users/odeli/Desktop/dataCamp_data/train_labels.csv")
train = pd.read_csv("C:/Users/odeli/Desktop/dataCamp_data/train.csv")

In [3]:
print('Shapes:\n - train: ', np.shape(train), '\n - train_labels: ',np.shape(train_labels), '\n - test: ',np.shape(test), '\n - specs: ',np.shape(specs), '\n - sample_submission: ',np.shape(sample_submission))

Shapes:
 - train:  (11341042, 11) 
 - train_labels:  (17690, 7) 
 - test:  (1156414, 11) 
 - specs:  (386, 3) 
 - sample_submission:  (1000, 2)


## Making labels

The first very important thing to do is to create the labels.
As we said the right information is contained in the event_data column.
This column is a dictionary with a variable size for each line.

Organisation of the information recorded by the application:
    * When a child downloads the app an id is created: id_installation
    * When he starts playing a game an other id is created: game_session
    * Inside a game_session many things can be done by the gamer. Every event has its own id: event_id. The different possible events are caracterized by a code contained in the event_code column.
The only interesting events to build the labels are the lasts one in a game session: they correspond to the succes or the failure of the game in a given game_session. The corrresponding event_code is 4100 for 4 of the 5 games and 4110 for the last one.

In [4]:
train[train['title'].str.contains('(Assessment)')].groupby('title').count()

C:\Users\odeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,type,world
title,,,,,,,,,,
Bird Measurer (Assessment),190164,190164,190164,190164,190164,190164,190164,190164,190164,190164
Cart Balancer (Assessment),163343,163343,163343,163343,163343,163343,163343,163343,163343,163343
Cauldron Filler (Assessment),181925,181925,181925,181925,181925,181925,181925,181925,181925,181925
Chest Sorter (Assessment),190318,190318,190318,190318,190318,190318,190318,190318,190318,190318
Mushroom Sorter (Assessment),199595,199595,199595,199595,199595,199595,199595,199595,199595,199595


In [19]:
train[train['title'].str.contains('Bird Measurer')].groupby('event_code').count()

,event_id,game_session,timestamp,event_data,installation_id,event_count,game_time,title,type,world
event_code,,,,,,,,,,
2000,3113,3113,3113,3113,3113,3113,3113,3113,3113,3113
2010,1430,1430,1430,1430,1430,1430,1430,1430,1430,1430
2020,4951,4951,4951,4951,4951,4951,4951,4951,4951,4951
2030,3275,3275,3275,3275,3275,3275,3275,3275,3275,3275
3010,10498,10498,10498,10498,10498,10498,10498,10498,10498,10498
3020,10009,10009,10009,10009,10009,10009,10009,10009,10009,10009
3021,1860,1860,1860,1860,1860,1860,1860,1860,1860,1860
3110,10269,10269,10269,10269,10269,10269,10269,10269,10269,10269
3120,9819,9819,9819,9819,9819,9819,9819,9819,9819,9819


In [22]:
tmp = train[train['title'].str.contains('Bird Measurer')]
np.shape(tmp)

(190164, 11)

In [23]:
tmp = tmp[tmp['event_code'] == 4110]
np.shape(tmp)

(11158, 11)

In [223]:
def get_accuracy(data, game, code):
    
    # Filtre par 1 des 5 jeux (game)
    tmp = data[data['title'].str.contains(game)]
    
    # Filtre dernier event = assessment 4110/4100 (code)
    tmp = tmp[tmp['event_code'] == code]
    
    # num_correct and num_incorrect
    correct = ["NA" for i in range(np.shape(tmp)[0])]
    incorrect = ["NA" for i in range(np.shape(tmp)[0])]
    for i in range(np.shape(tmp)[0]):
        if ('correct":false' in tmp.loc[tmp.index[i], 'event_data']):
            correct[i] = 0
            incorrect[i] = 1
        elif ('correct":true' in tmp.loc[tmp.index[i], 'event_data']):
            correct[i] = 1
            incorrect[i] = 0
        else:
            correct[i] = 'NA'
            incorrect[i] = 'NA'
    tmp['num_correct'] = correct
    tmp['num_incorrect'] = incorrect
    tmp = pd.DataFrame(tmp.groupby(('installation_id','game_session','title')).sum())
            
    # accuracy
    accuracy = tmp['num_correct'] / (tmp['num_correct'] + tmp['num_incorrect'])
    tmp['accuracy'] = accuracy
    
    # accuracy_group
    tmp["accuracy_group"] = tmp["accuracy"].apply(lambda x: 0 if x==0 else (1 if x<0.5 else (2 if x<0.9 else 3)))
    return(tmp)
    

In [249]:
BM_labels = get_accuracy(train, 'Bird Measurer', 4110)
CB_labels = get_accuracy(train, 'Cart Balancer', 4100)
CF_labels = get_accuracy(train, 'Cauldron Filler', 4100)
CS_labels = get_accuracy(train, 'Chest Sorter', 4100)
MS_labels = get_accuracy(train, 'Mushroom Sorter', 4100)

C:\Users\odeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.


In [250]:
my_train_labels = pd.concat([BM_labels,CB_labels,CF_labels,CS_labels,MS_labels])

In [252]:
np.shape(my_train_labels), np.shape(train_labels)

((17690, 7), (17690, 7))